# Feature Backfill Pipeline: KP, Solar Wind and Weather

This notebook builds a reproducible feature backfill pipeline for geomagnetic activity modeling.
It ingests, cleans, validates, and stores historical features from multiple physical domains:

- Geomagnetic indices (KP)
- Solar wind and IMF parameters (OMNI / CDAS)
- Terrestrial weather data (Sweden)

Each data source is ingested independently using its natural temporal coverage and later aligned
through feature views. The output of this notebook is a set of versioned feature groups suitable
for training and inference workflows.

In [1]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('aurora',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: C:\Users\lppap\Documents\master\scalable_ML\id2223-project
Added the following directory to the PYTHONPATH: C:\Users\lppap\Documents\master\scalable_ML\id2223-project
HopsworksSettings initialized!


## Imports

In [2]:
import datetime
import requests
import pandas as pd
import hopsworks
from mlfs.aurora import util
from cdasws import CdasWs
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

## Hopsworks login & Connect

In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 

2026-01-03 18:54:48,855 INFO: Initializing external client
2026-01-03 18:54:48,858 INFO: Base URL: https://c.app.hopsworks.ai:443


2026-01-03 18:54:52,760 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279154


## DB1 – KP Historical Data

This section ingests historical KP index data, which represents global geomagnetic activity.

KP is used as a core explanatory and/or target variable in geomagnetic and auroral models.
The data is treated as a daily-resolution, validated historical source suitable for training.

### Objective

Load, clean, and validate historical KP data and persist it as a standalone feature group.
This dataset defines the temporal backbone for downstream feature alignment.


### 1. Data Retrival

In [4]:
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=5)

csv_file = f"{root_dir}/data/kpdata.csv"
util.check_file_path(csv_file)

kp_yesterday_df = util.get_kp(csv_file, yesterday)
kp_yesterday_df.head()


File successfully found at the path: C:\Users\lppap\Documents\master\scalable_ML\id2223-project/data/kpdata.csv


,date,Kp1,Kp2,Kp3,Kp4,Kp5,Kp6,Kp7,Kp8,ap1,ap2,ap3,ap4,ap5,ap6,ap7,ap8,Ap
2189,2025-12-29,2.667,2.667,2.333,2.667,1.667,1.0,0.333,1.667,12.0,12.0,9.0,12.0,6.0,4.0,2.0,6.0,8.0


In [5]:
df = pd.read_csv(csv_file, skipinitialspace=True)

# Construct date from YYYY, MM, DD
df["date"] = pd.to_datetime(
    dict(year=df.YYYY, month=df.MM, day=df.DD)
)
#df

### 2. Cleaning & Resampling

Ensure numeric types of the values that we find interesting

#### Temporal Coverage and Integrity

Before using the KP data, we explicitly measure its temporal coverage and detect missing days.
This avoids silent gaps that could propagate into model training or evaluation.


In [6]:
df_kp = df[
    [
        "date",
        "Kp1", "Kp2", "Kp3", "Kp4", "Kp5", "Kp6", "Kp7", "Kp8",
        "ap1", "ap2", "ap3", "ap4", "ap5", "ap6", "ap7", "ap8",
        "Ap"
    ]
].copy()

# Ensure numeric types
feature_cols = [c for c in df_kp.columns if c != "date"]
df_kp[feature_cols] = df_kp[feature_cols].astype("float32")

# drop NaN values
df_kp.dropna(inplace=True)

#df_kp

### 3. (KP) Aurora-specific validation suite for geomagnetic features

In [7]:
import great_expectations as ge

kp_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="geomagnetic_expectation_suite"
)

for kp_col in ["kp1", "kp2", "kp3", "kp4", "kp5", "kp6", "kp7", "kp8"]:
    kp_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_between",
            kwargs={
                "column": kp_col,
                "min_value": 0.0,
                "max_value": 9.0
            }
        )
    )

for ap_col in ["ap1", "ap2", "ap3", "ap4", "ap5", "ap6", "ap7", "ap8", "ap"]:
    kp_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_between",
            kwargs={
                "column": ap_col,
                "min_value": 0.0,
                "max_value": 400.0
            }
        )
    )

### 4. Feature Group Creation and Insertion
1. Define Feature Group
2. Insert Data
3. Add description

In [8]:
df_kp.columns = df_kp.columns.str.lower()

# Define Feature Group
geomagnetic_fg = fs.get_or_create_feature_group(
    name="geomagnetic_daily",
    description="Daily global geomagnetic activity indices (Kp, ap, Ap)",
    version=1,
    primary_key=["date"],
    event_time="date",
    expectation_suite=kp_expectation_suite
)

# 2. Insert Data
geomagnetic_fg.insert(df_kp, wait=True)

2026-01-03 18:55:58,035 INFO: 	17 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279154/fs/1265765/fg/1876541


Uploading Dataframe: 100.00% |███████████████████████████| Rows 2190/2190 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: geomagnetic_daily_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279154/jobs/named/geomagnetic_daily_1_offline_fg_materialization/executions
2026-01-03 18:56:18,071 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2026-01-03 18:56:21,290 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-03 18:58:07,892 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2026-01-03 18:58:11,090 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-03 18:58:11,290 INFO: Waiting for log aggregation to finish.
2026-01-03 18:58:20,029 INFO: Execution finished successfully.


(Job('geomagnetic_daily_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_values_to_be_between",
         "kwargs": {
           "column": "kp8",
           "min_value": 0.0,
           "max_value": 9.0
         },
         "meta": {
           "expectationId": 799802
         }
       },
       "result": {
         "element_count": 2190,
         "missing_count": 0,
         "missing_percent": 0.0,
         "unexpected_count": 0,
         "unexpected_percent": 0.0,
         "unexpected_percent_total": 0.0,
         "unexpected_percent_nonmissing": 0.0,
         "partial_unexpected_list": []
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2026-01-03T05:55:58.000033Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": 

In [9]:
# 3. Add descriptions
geomagnetic_fg.update_feature_description(
    "date",
    "Date of global geomagnetic observation (daily resolution)"
)

geomagnetic_fg.update_feature_description(
    "kp1",
    "Geomagnetic Kp index for 00:00–03:00 UTC"
)
geomagnetic_fg.update_feature_description(
    "kp2",
    "Geomagnetic Kp index for 03:00–06:00 UTC"
)
geomagnetic_fg.update_feature_description(
    "kp3",
    "Geomagnetic Kp index for 06:00–09:00 UTC"
)
geomagnetic_fg.update_feature_description(
    "kp4",
    "Geomagnetic Kp index for 09:00–12:00 UTC"
)
geomagnetic_fg.update_feature_description(
    "kp5",
    "Geomagnetic Kp index for 12:00–15:00 UTC"
)
geomagnetic_fg.update_feature_description(
    "kp6",
    "Geomagnetic Kp index for 15:00–18:00 UTC"
)
geomagnetic_fg.update_feature_description(
    "kp7",
    "Geomagnetic Kp index for 18:00–21:00 UTC"
)
geomagnetic_fg.update_feature_description(
    "kp8",
    "Geomagnetic Kp index for 21:00–24:00 UTC"
)

geomagnetic_fg.update_feature_description(
    "ap1",
    "Linear ap geomagnetic index corresponding to kp1"
)
geomagnetic_fg.update_feature_description(
    "ap2",
    "Linear ap geomagnetic index corresponding to kp2"
)
geomagnetic_fg.update_feature_description(
    "ap3",
    "Linear ap geomagnetic index corresponding to kp3"
)
geomagnetic_fg.update_feature_description(
    "ap4",
    "Linear ap geomagnetic index corresponding to kp4"
)
geomagnetic_fg.update_feature_description(
    "ap5",
    "Linear ap geomagnetic index corresponding to kp5"
)
geomagnetic_fg.update_feature_description(
    "ap6",
    "Linear ap geomagnetic index corresponding to kp6"
)
geomagnetic_fg.update_feature_description(
    "ap7",
    "Linear ap geomagnetic index corresponding to kp7"
)
geomagnetic_fg.update_feature_description(
    "ap8",
    "Linear ap geomagnetic index corresponding to kp8"
)

geomagnetic_fg.update_feature_description(
    "ap",
    "Daily average linear geomagnetic ap index"
)

## DB2 - Weather Data
This section ingests historical weather data for Sweden.

Weather variables provide terrestrial context that may influence visibility conditions,
measurement reliability, or secondary correlations with geomagnetic activity.

### Data Source and Scope

Weather data is retrieved for a fixed geographic location using a historical API.
The dataset is aggregated to daily resolution to match the temporal granularity
of KP and solar wind features.


### 1. Data Retrival

In [13]:
earliest_kp_date = df_kp["date"].min().strftime("%Y-%m-%d")
end_date = (datetime.date.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

latitude = 62.0
longitude = 15.0

weather_df = util.get_historical_weather_sweden(
    earliest_kp_date,
    end_date,
    latitude,
    longitude
)

weather_df.head()


,date,cloud_cover_mean,precipitation_sum,sunshine_duration
0,2020-01-01,92.333336,0.0,3612.911377
1,2020-01-02,96.625000,0.0,0.000000
2,2020-01-03,52.958332,0.0,7516.577148
3,2020-01-04,19.500000,0.0,8072.493164
4,2020-01-05,77.000000,1.0,0.000000


In [14]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2194 entries, 0 to 2193
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               2194 non-null   datetime64[ns]
 1   cloud_cover_mean   2194 non-null   float32       
 2   precipitation_sum  2194 non-null   float32       
 3   sunshine_duration  2194 non-null   float32       
dtypes: datetime64[ns](1), float32(3)
memory usage: 43.0 KB


### 3. Weather validation suite

In [15]:
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="sweden_weather_expectation_suite"
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "cloud_cover_mean",
            "min_value": 0.0,
            "max_value": 100.0
        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "precipitation_sum",
            "min_value": 0.0
        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "sunshine_duration",
            "min_value": 0.0
        }
    )
)

{"expectation_type": "expect_column_values_to_be_between", "kwargs": {"column": "sunshine_duration", "min_value": 0.0}, "meta": {}}

### 4. Feature Group Creation and Insertion

In [16]:
weather_fg = fs.get_or_create_feature_group(
    name="sweden_weather_daily",
    description="Daily weather conditions in Sweden relevant for aurora observability",
    version=1,
    primary_key=["date"],
    event_time="date",
    expectation_suite=weather_expectation_suite
)

weather_fg.insert(weather_df, wait=True)

2026-01-03 19:02:36,342 INFO: 	3 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279154/fs/1265765/fg/1876542


Uploading Dataframe: 100.00% |███████████████████████████| Rows 2194/2194 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: sweden_weather_daily_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279154/jobs/named/sweden_weather_daily_1_offline_fg_materialization/executions
2026-01-03 19:02:56,736 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2026-01-03 19:03:06,376 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-03 19:04:46,300 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-03 19:04:46,477 INFO: Waiting for log aggregation to finish.
2026-01-03 19:04:55,210 INFO: Execution finished successfully.


(Job('sweden_weather_daily_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_values_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": 0.0
         },
         "meta": {
           "expectationId": 799812
         }
       },
       "result": {
         "element_count": 2194,
         "missing_count": 0,
         "missing_percent": 0.0,
         "unexpected_count": 0,
         "unexpected_percent": 0.0,
         "unexpected_percent_total": 0.0,
         "unexpected_percent_nonmissing": 0.0,
         "partial_unexpected_list": []
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2026-01-03T06:02:36.000341Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       

In [17]:
weather_fg.update_feature_description(
    "date",
    "Date of daily weather observation for Sweden"
)

weather_fg.update_feature_description(
    "cloud_cover_mean",
    "Mean daily cloud cover in percent (0–100), affecting aurora visibility"
)

weather_fg.update_feature_description(
    "precipitation_sum",
    "Total daily precipitation in millimeters (rain or snow)"
)

weather_fg.update_feature_description(
    "sunshine_duration",
    "Total duration of sunshine during the day in seconds, proxy for sky clarity"
)

## DB3 - Solar wind & IMF Hisorical Data Feature Group
This section ingests solar wind and interplanetary magnetic field (IMF) data
from NASA's OMNI dataset via the CDAS API.

Solar wind parameters provide upstream physical drivers of geomagnetic activity.

### 1. Data Retrival

In [18]:
from cdasws import CdasWs
cdas = CdasWs()

# Search for datasets with "OMNI" in the ID and "Hourly" in the label
datasets = cdas.get_datasets(idPattern='AC_H1_MFI')

for index, dataset in enumerate(datasets):
    print(f"{index}. ID: {dataset['Id']} | Label: {dataset['Label']}")

0. ID: AC_H1_MFI | Label: H1 - ACE Magnetic Field 4-Minute Level 2 Data - N. Ness (Bartol Research Institute)


In [19]:
# Use the ID we found or the one you were using
ds_id = 'OMNI2_H0_MRG1HR' 

variables = cdas.get_variables(ds_id)

print(f"--- Variables for {ds_id} ---")
for var in variables:
    # 'Name' is what you put in your variables list
    # 'LongDescription' tells you what the physics is
    print(f"Variable ID: {var['Name']:15} | Description: {var['LongDescription']}")


--- Variables for OMNI2_H0_MRG1HR ---
Variable ID: Rot1800         | Description: Bartels Rotation Number (half-hour midpoint)
Variable ID: IMF1800         | Description: OMNI ID code for the source spacecraft for time-shifted IMF values (see OMNI documentation link for codes) (half-hour midpoint)
Variable ID: PLS1800         | Description: OMNI ID code for the source spacecraft for time-shifted IP plasma values (see OMNI documentation link for codes) (half-hour midpoint)
Variable ID: IMF_PTS1800     | Description: # fine time scale IMF points (half-hour midpoint)
Variable ID: PLS_PTS1800     | Description: # fine time scale plasma points (half-hour midpoint)
Variable ID: ABS_B1800       | Description: 1AU IP Average B Field Magnitude, nT, (last currently-available OMNI B-field data Nov 24, 2025) (half-hour midpoint)
Variable ID: F1800           | Description: 1AU IP Magnitude of average field vector (nT) (half-hour midpoint)
Variable ID: THETA_AV1800    | Description: 1AU IP Latitude/

In [34]:
from cdasws.datarepresentation import DataRepresentation as dr

cdas = CdasWs()

# Parameters from NASA's official OMNI2 dataset
dataset = 'OMNI2_H0_MRG1HR'
# V1800: Speed, BZ_GSM1800: Magnetic Field, N1800: Density, R1800: Sunspots
variables = ['V1800', 'BZ_GSM1800', 'N1800', 'R1800', 'Pressure1800'] 
start_time = '2023-01-01T00:00:00Z'
end_time = datetime.datetime.now(datetime.timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')

status, data = cdas.get_data(dataset, variables, start_time, end_time, dataRepresentation=dr.SPACEPY)
# 2. Fetch the data
status, data = cdas.get_data(dataset, variables, start_time, end_time)

# 3. Handle the response and convert to Pandas
if status['http']['status_code'] == 200 and data is not None:
    physics_df = pd.DataFrame({
        'date': pd.to_datetime(data['Epoch']),
        'vsw': data['V1800'],
        'bz': data['BZ_GSM1800'],
        'density': data['N1800'],
        'pressure': data['Pressure1800'],
        'sunspots': data['R1800']
    })

    # 4. Clean and Resample to Daily
    # NASA's missing data flag for these variables is usually 999.9 or 999.99
    physics_df = physics_df.replace([999.9, 999.99, 9999., 99.99], pd.NA).ffill()
    physics_df['date'] = pd.to_datetime(data['Epoch'])

    last_available_date = physics_df['date'].max()

    print("📌 Última fecha disponible en OMNI:", last_available_date)

    physics_daily = physics_df.resample('D', on='date').mean().reset_index()

    print("✅ Data successfully retrieved!")
    print(len(physics_daily))
    
else:
    print(f"❌ Failed to retrieve data. Status code: {status}")
    print("If status is 400, verify variable names by running: cdas.get_variables('OMNI2_H0_MRG1HR')")

📌 Última fecha disponible en OMNI: 2025-12-26 12:00:00
✅ Data successfully retrieved!
1091


In [35]:
 for lag in [1, 2, 3]:
    physics_daily[f'vsw_lag{lag}'] = physics_daily['vsw'].shift(lag)
    physics_daily[f'bz_lag{lag}'] = physics_daily['bz'].shift(lag)
    physics_daily[f'pressure_lag{lag}'] = physics_daily['pressure'].shift(lag)

physics_daily['bz_3d_mean'] = physics_daily['bz'].rolling(3).mean()
physics_daily['bz_7d_min'] = physics_daily['bz'].rolling(7).min()

physics_daily['vsw_3d_mean'] = physics_daily['vsw'].rolling(3).mean()
physics_daily['pressure_3d_max'] = physics_daily['pressure'].rolling(3).max()

physics_daily['vBz'] = physics_daily['vsw'] * physics_daily['bz']
physics_daily['vBz_neg'] = physics_daily['vsw'] * physics_daily['bz'].clip(upper=0)

float_cols = physics_daily.select_dtypes("float").columns
physics_daily[float_cols] = physics_daily[float_cols].astype("float32")

before = len(physics_daily)

physics_daily = physics_daily.dropna().reset_index(drop=True)

after = len(physics_daily)

print(f"🧹 Dropped {before - after} rows due to NaNs")
print(f"📊 Remaining rows: {after}")


🧹 Dropped 6 rows due to NaNs
📊 Remaining rows: 1085


In [36]:
physics_daily.head()

,date,vsw,bz,density,pressure,sunspots,vsw_lag1,bz_lag1,pressure_lag1,vsw_lag2,...,pressure_lag2,vsw_lag3,bz_lag3,pressure_lag3,bz_3d_mean,bz_7d_min,vsw_3d_mean,pressure_3d_max,vBz,vBz_neg
0,2023-01-07,418.666656,2.058333,4.233333,1.435000,133.0,389.583344,3.950000,0.918750,364.791656,...,3.033750,406.000000,4.329167,3.434583,2.881944,-0.391667,391.013885,3.033750,861.755554,0.000000
1,2023-01-08,388.875000,-0.495833,751.183350,75.374580,135.0,418.666656,2.058333,1.435000,389.583344,...,0.918750,364.791656,2.637500,3.033750,1.837500,-0.495833,399.041656,75.374580,-192.817200,-192.817200
2,2023-01-09,370.458344,-0.220833,503.008331,50.682919,148.0,388.875000,-0.495833,75.374580,418.666656,...,1.435000,389.583344,3.950000,0.918750,0.447222,-0.495833,392.666656,75.374580,-81.809555,-81.809555
3,2023-01-10,363.166656,0.529167,9.654166,2.572500,179.0,370.458344,-0.220833,50.682919,388.875000,...,75.374580,418.666656,2.058333,1.435000,-0.062500,-0.495833,374.166656,75.374580,192.175705,0.000000
4,2023-01-11,387.625000,-0.587500,7.995833,2.408750,168.0,363.166656,0.529167,2.572500,370.458344,...,50.682919,388.875000,-0.495833,75.374580,-0.093056,-0.587500,373.750000,50.682919,-227.729675,-227.729675


### 3. Solar Data validation Ensuit

In [48]:
solar_wind_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="solar_wind_nasa_expectation_suite"
)

# Check columns exist
for col in ["vsw", "bz", "density", "pressure", "sunspots", "date"]:
    solar_wind_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_to_exist",
            kwargs={"column": col}
        )
    )

# valid time stamp
solar_wind_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_null",
        kwargs={"column": "date"}
    )
)

solar_wind_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "vsw",
            "min_value": 200.0,
            "max_value": 2000.0
        }
    )
)

solar_wind_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "bz",
            "min_value": -100.0,
            "max_value": 100.0
        }
    )
)

solar_wind_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "density",
            "min_value": 0.01,
            "max_value": 500.0
        }
    )
)

solar_wind_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "pressure",
            "min_value": 0.0,
            "max_value": 100.0
        }
    )
)

solar_wind_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "sunspots",
            "min_value": 0.0,
            "max_value": 500.0
        }
    )
)

{"expectation_type": "expect_column_values_to_be_between", "kwargs": {"column": "sunspots", "min_value": 0.0, "max_value": 500.0}, "meta": {}}

In [49]:
derived_columns = [
    "vsw_lag1", "vsw_lag2", "vsw_lag3",
    "bz_lag1", "bz_lag2", "bz_lag3",
    "pressure_lag1", "pressure_lag2", "pressure_lag3",
    "bz_3d_mean", "bz_7d_min",
    "vsw_3d_mean", "pressure_3d_max",
    "vbz", "vbz_neg"
]

for col in derived_columns:
    solar_wind_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_to_exist",
            kwargs={"column": col}
        )
    )

### 4. Feature Group Creation and Insertion

In [50]:
solar_wind_fg = fs.get_or_create_feature_group(
    name="nasa_omni_daily",
    description="NASA's OMNI daily data information about solar wind",
    version=1,
    primary_key=["date"],
    event_time="date",
    expectation_suite=solar_wind_expectation_suite
)

solar_wind_fg.insert(physics_daily, wait=True)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279154/fs/1265765/fg/1890681
2026-01-03 19:36:33,907 INFO: 	27 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279154/fs/1265765/fg/1890681


Uploading Dataframe: 100.00% |███████████████████████████| Rows 1085/1085 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: nasa_omni_daily_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279154/jobs/named/nasa_omni_daily_1_offline_fg_materialization/executions
2026-01-03 19:36:52,151 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-03 19:36:55,335 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-03 19:38:40,214 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-03 19:38:40,381 INFO: Waiting for log aggregation to finish.
2026-01-03 19:38:48,968 INFO: Execution finished successfully.


(Job('nasa_omni_daily_1_offline_fg_materialization', 'SPARK'),
 {
   "success": false,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_to_exist",
         "kwargs": {
           "column": "pressure_lag2"
         },
         "meta": {
           "expectationId": 803864
         }
       },
       "result": {},
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2026-01-03T06:36:33.000906Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_values_to_be_between",
         "kwargs": {
           "column": "bz",
           "min_value": -100.0,
           "max_value": 100.0
         },
         "meta": {
           "expectationId": 803861
         }
       },
       "re

In [53]:
feature_descriptions = {
    "vsw": "Daily mean solar wind speed measured near Earth (km/s).",
    "bz": "Daily mean z-component of the interplanetary magnetic field in GSM coordinates (nT).",
    "density": "Daily mean solar wind proton density (particles per cm^3).",
    "pressure": "Daily mean solar wind dynamic pressure (nPa).",
    "sunspots": "Daily international sunspot number, proxy for solar cycle activity.",
    "vsw_lag1": "Solar wind speed one day prior.",
    "vsw_lag2": "Solar wind speed two days prior.",
    "vsw_lag3": "Solar wind speed three days prior.",
    "bz_lag1": "IMF Bz one day prior.",
    "bz_lag2": "IMF Bz two days prior.",
    "bz_lag3": "IMF Bz three days prior.",
    "pressure_lag1": "Solar wind dynamic pressure one day prior.",
    "pressure_lag2": "Solar wind dynamic pressure two days prior.",
    "pressure_lag3": "Solar wind dynamic pressure three days prior.",
    "bz_3d_mean": "3-day rolling mean of IMF Bz, capturing sustained southward conditions.",
    "bz_7d_min": "Minimum IMF Bz over the previous 7 days, highlighting prolonged southward intervals.",
    "vsw_3d_mean": "3-day rolling mean of solar wind speed.",
    "pressure_3d_max": "Maximum solar wind dynamic pressure over the previous 3 days.",
    "vbz": "Product of solar wind speed and IMF Bz, proxy for solar wind–magnetosphere coupling.",
    "vbz_neg": "Coupling proxy considering only southward IMF Bz (negative values)."
}

In [55]:
for feature, description in feature_descriptions.items():
    solar_wind_fg.update_feature_description(feature, description)